<a href="https://colab.research.google.com/github/RajeshMallik19/Project_Q/blob/student_dropout/Student_4_Quantum_Model_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===================================================================
# Part 0: Install and Import Libraries
# ===================================================================
!pip install pennylane pennylane-lightning
!pip install imbalanced-learn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
import tensorflow as tf
import pennylane as qml

print("Libraries installed and imported.")


# ===================================================================
# Part 1: Load and Prepare Data
# ===================================================================
df = pd.read_csv('student_dropout_prepped.csv')
X = df.drop(columns=['at_risk'])
y = df['at_risk']
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Balanced 10-qubit model for reasonable runtime
num_components = 10
pca = PCA(n_components=num_components)
X_train_pca = pca.fit_transform(X_train_resampled)
X_test_pca = pca.transform(X_test)

# Convert data to TensorFlow Tensors
X_train_tf = tf.constant(X_train_pca, dtype=tf.float32)
y_train_tf = tf.constant(y_train_resampled, dtype=tf.float32)
X_test_tf = tf.constant(X_test_pca, dtype=tf.float32)
y_test_tf = tf.constant(y_test, dtype=tf.float32)

print("Student data loaded and prepped.")


# ===================================================================
# Part 2: Define the Quantum Circuit
# ===================================================================
num_qubits = num_components
dev = qml.device("lightning.qubit", wires=num_qubits)

@qml.qnode(dev, interface="tf")
def qnode(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(num_qubits))
    qml.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))


# ===================================================================
# Part 3: Define the Model and Variables
# ===================================================================
num_layers = 3
q_weights_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)
q_weights = tf.Variable(tf.random.normal(shape=q_weights_shape, dtype=tf.float32), trainable=True)
cl_weights = tf.Variable(tf.random.normal(shape=(1,), dtype=tf.float32), trainable=True)
cl_bias = tf.Variable(tf.zeros(shape=(1,), dtype=tf.float32), trainable=True)

def hybrid_model(x, q_weights, cl_weights, cl_bias):
    q_output = qnode(x, q_weights)
    q_output_float32 = tf.cast(q_output, dtype=tf.float32)
    return tf.sigmoid(q_output_float32 * cl_weights + cl_bias)

print("Hybrid model and variables defined.")

# ===================================================================
# Part 4: The Custom Training Loop
# ===================================================================
epochs = 10
batch_size = 16
num_batches = len(X_train_tf) // batch_size

optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss_fn = tf.keras.losses.BinaryCrossentropy()

print("\nStarting custom training loop for student data...")

for epoch in range(epochs):
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        x_batch = X_train_tf[start:end]
        y_batch = y_train_tf[start:end]

        with tf.GradientTape() as tape:
            predictions = hybrid_model(x_batch, q_weights, cl_weights, cl_bias)
            y_batch_reshaped = tf.reshape(y_batch, predictions.shape)
            loss = loss_fn(y_batch_reshaped, predictions)

        trainable_vars = [q_weights, cl_weights, cl_bias]
        gradients = tape.gradient(loss, trainable_vars)
        optimizer.apply_gradients(zip(gradients, trainable_vars))

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.numpy():.4f}")


print("\nTraining complete.")

# ===================================================================
# Part 5: Final Evaluation
# ===================================================================
print("\n--- Final Quantum Model Evaluation (Student Data) ---")
y_pred_proba = hybrid_model(X_test_tf, q_weights, cl_weights, cl_bias)
y_pred = (y_pred_proba > 0.5).numpy().astype(int)
print(classification_report(y_test, y_pred))

Libraries installed and imported.
Student data loaded and prepped.
Hybrid model and variables defined.

Starting custom training loop for student data...
Epoch 1/10, Loss: 0.7674
Epoch 2/10, Loss: 0.6573
Epoch 3/10, Loss: 0.5576
Epoch 4/10, Loss: 0.5092
Epoch 5/10, Loss: 0.4712
Epoch 6/10, Loss: 0.4415
Epoch 7/10, Loss: 0.4177
Epoch 8/10, Loss: 0.3976
Epoch 9/10, Loss: 0.3799
Epoch 10/10, Loss: 0.3637

Training complete.

--- Final Quantum Model Evaluation (Student Data) ---
              precision    recall  f1-score   support

           0       0.93      0.89      0.91        71
           1       0.71      0.80      0.75        25

    accuracy                           0.86        96
   macro avg       0.82      0.84      0.83        96
weighted avg       0.87      0.86      0.87        96

